# Implementation of U-Net in pytorch
## Import Packages

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Set device to CUDA if available

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device == 'cuda':
    print("Cuda available: " + torch.cuda.get_device_name())
else:
    print("Cuda unavailable.")


Cuda unavailable.


## Define basic block of U-Net

In [4]:
class downConv(nn.Module):
    def __init__(self, in_c, out_c, kernel_size=3, padding=1, stride=1,):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=kernel_size, padding=padding, stride=stride,),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, kernel_size=kernel_size, padding=padding, stride=stride,),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)



class upConv(nn.Module):
    def __init__(self, in_c, out_c, kernel_size=3, padding=1, stride=1,):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_c, in_c // 2, kernel_size=kernel_size, padding=padding, stride=stride,),
            nn.BatchNorm2d(in_c // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_c // 2, out_c, kernel_size=kernel_size, padding=padding, stride=stride,),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

## Define U-Net

In [5]:
class unet_encoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.block1 = downConv(in_c=3, out_c=64)
        self.block2 = downConv(in_c=64, out_c=128)
        self.block3 = downConv(in_c=128, out_c=256)
        self.block4 = downConv(in_c=256, out_c=512)
        self.block5 = downConv(in_c=512, out_c=1024)
        
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        x1 = self.block1(x)
        print(x1.shape)
        x2 = self.block2(self.pool(x1))
        print(x2.shape)
        x3 = self.block3(self.pool(x2))
        print(x3.shape)
        x4 = self.block4(self.pool(x3))
        print(x4.shape)
        x5 = self.block5(self.pool(x4))
        print(x5.shape)

        return [x1, x2, x3, x4, x5]

In [6]:
class unet_decoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.upconv1 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2, stride=2,)
        self.upconv2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2,)
        self.upconv3 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=2,)
        self.upconv4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2,)
        
        self.block1 = upConv(in_c=1024, out_c=512)
        self.block2 = upConv(in_c=512, out_c=256)
        self.block3 = upConv(in_c=256, out_c=128)
        self.block4 = upConv(in_c=128, out_c=64)

    def forward(self, x_set):
        x1_1, x1_2, x1_3, x1_4, x1_5 = x_set
        
        x2_1 = self.upconv1(x1_5)
        x2_1 = torch.cat([x2_1, x1_4], dim=1)
        x2_1 = self.block1(x2_1)
        print(x2_1.shape)
        
        x2_2 = self.upconv2(x2_1)
        x2_2 = torch.cat([x2_2, x1_3], dim=1)
        x2_2 = self.block2(x2_2)
        print(x2_2.shape)

        x2_3 = self.upconv3(x2_2)
        x2_3 = torch.cat([x2_3, x1_2], dim=1)
        x2_3 = self.block3(x2_3)
        print(x2_3.shape)

        x2_4 = self.upconv4(x2_3)
        x2_4 = torch.cat([x2_4, x1_1], dim=1)
        x2_4 = self.block4(x2_4)
        print(x2_4.shape)
        
        return x2_4

In [7]:
class u_net(nn.Module):
    def __init__(self):
        super().__init__()

        self.part1 = unet_encoder()
        self.part2 = unet_decoder()

        self.seg_head = nn.Sequential(
            nn.Conv2d(64, 2, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x1 = self.part1(x)
        x2 = self.part2(x1)

        x = self.seg_head(x2)

        return x

In [8]:
if __name__ == '__main__': 
    model = u_net()
    x = torch.randn(1, 3, 224, 224)
    output = model(x)
    print("Output Shape:", output.shape)
    

torch.Size([1, 64, 224, 224])
torch.Size([1, 128, 112, 112])
torch.Size([1, 256, 56, 56])
torch.Size([1, 512, 28, 28])
torch.Size([1, 1024, 14, 14])
torch.Size([1, 512, 28, 28])
torch.Size([1, 256, 56, 56])
torch.Size([1, 128, 112, 112])
torch.Size([1, 64, 224, 224])
Output Shape: torch.Size([1, 2, 224, 224])
